In [1]:
import json
import glob

In [13]:
input_folder = "../output/output_compare/"
# input_folder = "../output/"

files = glob.glob(input_folder + '*.json', recursive = False)

In [12]:
results_folder = "../notebooks/results/"
version = "0"

In [4]:
def compare_detected_fields(found_fields):
    pages_with_types = []
    pages_with_value_field = []
    pages_with_equal = []
    pages_with_less = []
    pages_with_more = []
    pages_with_no_fields = []

    for page, data in found_fields.items():
        has_fields = False

        if data.get("count_fields_from_type") != 0:
            pages_with_types.append(page)
            has_fields = True
        if data.get("count_fields_with_value") > data.get("count_fields_without_value"):
            pages_with_value_field.append(page)
            has_fields = True

        if data.get("comparison") == "equal" and data.get("count_fields_from_fathom") != 0:
            pages_with_equal.append(page)
            has_fields = True
        elif data.get("comparison") == "less":
            pages_with_less.append(page)
            has_fields = True
        elif data.get("comparison") == "more":
            pages_with_more.append(page)
            has_fields = True
        
        if not has_fields:
            pages_with_no_fields.append(page)
    
    return pages_with_types, pages_with_value_field, pages_with_equal, pages_with_less, pages_with_more, pages_with_no_fields

In [14]:
failed_sites = []
sites_with_login_detected = []

sites_with_types = []
sites_with_value = []
sites_with_equal = []
sites_with_less = []
sites_with_more = []
sites_with_no_fields = []

total_pages = 0

total_pages_with_types = []
total_pages_with_value = []
total_pages_with_equal = []
total_pages_with_less = []
total_pages_with_more = []
total_pages_with_no_fields = []

for file in files:
    try:
        # Skip general log file
        if file == input_folder + "log.json":
            continue

        site_name = file
        # print(site_name)
        site_name = site_name[len(input_folder):]
        site_name = site_name[:-len(".json")]

        if site_name.startswith("www."):
            site_name = site_name[4:]
        
        # print(site_name)

        # Load the data from the .json file
        f = open(file, encoding="utf-8") 
        data = json.load(f)
        f.close()

        found_fields = data.get("found_fields")
        if found_fields is None or len(found_fields) == 0:
            continue
        sites_with_login_detected.append(site_name)
        total_pages += len(found_fields)

        # Process results
        pages_with_types, pages_with_value_field, pages_with_equal, pages_with_less, pages_with_more, pages_with_no_fields = compare_detected_fields(found_fields)

        if len(pages_with_types):
            sites_with_types.append(site_name)
            total_pages_with_types += pages_with_types
        if len(pages_with_value_field):
            sites_with_value.append(site_name)
            total_pages_with_value += pages_with_value_field
        if len(pages_with_equal):
            sites_with_equal.append(site_name)
            total_pages_with_equal += pages_with_equal
        if len(pages_with_less):
            sites_with_less.append(site_name)
            total_pages_with_less += pages_with_less
        if len(pages_with_more):
            sites_with_more.append(site_name)
            total_pages_with_more += pages_with_more
        if len(pages_with_no_fields):
            sites_with_no_fields.append(site_name)
            total_pages_with_no_fields += pages_with_no_fields

    except:
        failed_sites.append(file)

results = {
    "count_total_sites": len(files),
    "sites_with_login_detected": sites_with_login_detected,
    "count_sites_with_login_detected": len(sites_with_login_detected),
    "sites_with_types": sites_with_types,
    "count_sites_with_types": len(sites_with_types),
    "sites_with_value": sites_with_value,
    "count_sites_with_value": len(sites_with_value),
    "sites_with_equal": sites_with_equal,
    "count_sites_with_equal": len(sites_with_equal),
    "sites_with_less": sites_with_less,
    "count_sites_with_less": len(sites_with_less),
    "sites_with_more": sites_with_more,
    "count_sites_with_more": len(sites_with_more),
    "sites_with_no_fields": sites_with_no_fields,
    "count_sites_with_no_fields": len(sites_with_no_fields),
    "failed_sites_in_processing": failed_sites,
    "count_failed_sites_in_processing": len(failed_sites)
}

print(len(files))
print(len(failed_sites))

with open(results_folder + "results_" + version + ".json", 'w', encoding='utf-8') as fp:
        json.dump(results, fp, default=tuple, sort_keys=True, indent=4)
        fp.close()

950
0


In [15]:
page_results = {
    "count_total_sites": len(files),
    "count_total_pages": total_pages,
    "pages_with_types": total_pages_with_types,
    "count_pages_with_types": len(total_pages_with_types),
    "pages_with_value": total_pages_with_value,
    "count_pages_with_value": len(total_pages_with_value),
    "pages_with_equal": total_pages_with_equal,
    "count_pages_with_equal": len(total_pages_with_equal),
    "pages_with_less": total_pages_with_less,
    "count_pages_with_less": len(total_pages_with_less),
    "pages_with_more": total_pages_with_more,
    "count_pages_with_more": len(total_pages_with_more),
    "pages_with_no_fields": total_pages_with_no_fields,
    "count_pages_with_no_fields": len(total_pages_with_no_fields)
}

with open(results_folder + "page_results_" + version + ".json", 'w', encoding='utf-8') as fp:
        json.dump(page_results, fp, default=tuple, sort_keys=True, indent=4)
        fp.close()